In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision import models
from torchvision.models.resnet import resnet34
from torchvision.transforms.transforms import Resize
import torch.nn.functional as F
import json
import numpy as np
from PIL import Image

if torch.cuda.is_available():
        device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

transform = transforms.Compose([
transforms.Resize(224),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
#采用自带的Cifar100
trainset = torchvision.datasets.CIFAR100(root='./data_CIFAR100', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
 
testset = torchvision.datasets.CIFAR100(root='./data_CIFAR100', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


### 在数据集中加入weight

In [3]:
from torchvision import datasets
class CIFAR100(datasets.CIFAR100):
    def __init__(self, root, indexs, influence_weight, train=True,
                 transform=torchvision.transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]),
                 download=True):
        super().__init__(root, train=train,
                         transform=transform,
                         download=download)
        if indexs is not None:
            self.data = self.data[indexs]
            self.targets = np.array(self.targets)[indexs]
            self.index = indexs
            self.weight = influence_weight[indexs]


    def __getitem__(self, index):
        img, targets,indexs,weight = self.data[index], \
                                    self.targets[index], \
                                    self.index[index],\
                                    self.weight[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img, targets,indexs,weight

In [4]:
import json
import numpy as np
def read_json(save_path, encoding='utf8'):
    with open(save_path, 'r', encoding=encoding) as f:
        content = f.read()
        content = json.loads(content)
        return content

def normalize(x, newRange=(0, 1)): #x is an array. Default range is between zero and one
    xmin, xmax = np.min(x), np.max(x) #get max and min from input array
    norm = (x - xmin)/(xmax - xmin) # scale between zero and one

    if newRange == (0, 1):
        return(norm) # wanted range is the same as norm
    elif newRange != (0, 1):
        return norm * (newRange[1] - newRange[0]) + newRange[0] #scale to a different range.

In [7]:

influence_0 = read_json("./influence_results_tmp_0_index_9.json")['0']['influence']
# influence_1 = read_json(save_path)['1']['influence']
# influence_2 = read_json(save_path)['2']['influence']
# influence_3 = read_json(save_path)['3']['influence']
# influence_4 = read_json(save_path)['4']['influence']

# influence = [(x0+x1+x2+x3+x4)/5
#              for x0,x1,x2,x3,x4
#              in zip(influence_0,influence_1,influence_2,influence_3,influence_4)
#             ]

normalized_influence_score = normalize(influence_0, newRange=(-1, 1))

influence_weight = np.exp(normalized_influence_score)


In [9]:
all_idx = np.array(range(len(trainset.targets)))
default_weight = np.ones(len(trainset.targets))

train_dataset = CIFAR100(root='./data_CIFAR100',
     indexs = all_idx,influence_weight=influence_weight, train=True
)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)


Files already downloaded and verified


In [ ]:
## 训练过程与KD_with_IF中的一样